In [9]:
import os
import re
import pymupdf
import django_setup
from IPython.display import Markdown, display
from bs4 import BeautifulSoup as bs

from django.conf import settings

from cmj.utils import clean_text

gemini_api_key = settings.GEMINI_API_KEY

In [10]:
import google.generativeai as genai
genai.configure(api_key=gemini_api_key)


In [23]:
def get_model_configured():
    generation_config = {
      "temperature": 0.1,
      "top_p": 0.95,
      "top_k": 40,
      #"max_output_tokens": 8192,
      "response_mime_type": "text/plain",
    }

    model = genai.GenerativeModel(
      model_name="gemini-2.0-flash-exp",
      #model_name="gemini-2.5-pro-exp-03-25",
      #model_name="gemini-2.5-flash-preview-04-17",
      generation_config=generation_config,
    )

    return model

model = get_model_configured()


In [24]:
def make_prompt(num_prompt, original, analisado, o_epigrafe, a_epigrafe):

# criar um prompt para o modelo generativo para comparar dois textos
# o primeiro texto é o original e o segundo é o analisado
# o modelo deve comparar os dois textos e responder as perguntas
# o modelo deve ser capaz de identificar se os dois textos estão pedindo o mesmo benefício


    prompt0 = f"""
Assuma a personalidade de um especialista em produção de textos legislativos em uma câmara municipal,
com formação em direito e experiência em redação de documentos oficiais. Sua tarefa é analisar dois textos
que são pedidos de providências feitos por vereadores e identificar se eles estão pedindo o mesmo benefício.
Os textos podem ter diferenças de redação, mas você deve se concentrar no conteúdo e na intenção dos pedidos.
Para tal tarefa, compare o conteúdo de <ORIGINAL></ORIGINAL> com o conteúdo de <ANALISADO></ANALISADO>.
Para citar estes dois conteúdos, nomeie eles respectivamente da seguinte maneira:
"{o_epigrafe}" e "{a_epigrafe}".
Remova de sua análise os autores e os destinatários pois são irrelevantes para a comparação requerida.
O importante é o que está sendo pedido, quem será o beneficiário do pedido e para qual localidade
está sendo feito tal pedido.
Não faça considerações adicionais e ou mesmo conclusão extra. Neste contexto responda:

    - Os textos estão pedindo o mesmo benefício para a mesma localidade?
    - Descreva de forma sucinta e direta o que está sendo pedido em <ORIGINAL></ORIGINAL> e <ANALISADO></ANALISADO> informando também os beneficiários e as localidades citadas.
    - Calcule a semelhança percentual entre os documentos desconsiderando autores e destinatários, focando na solicitação e no beneficiário, qual semelhança percentual entre <ORIGINAL></ORIGINAL> e <ANALISADO></ANALISADO>? Coloque o resultado em percentual com uma marcação fácil de ser extraída via regex "\[\[ \d{1,3}% \]\]".
    - formate a resposta em MARKDOWN, utilizando linguagem dissertativa com os títulos e subtítulos necessários para facilitar a leitura, utilizando negrito e itálico quando necessário



<ORIGINAL>{original}</ORIGINAL>

<ANALISADO>{analisado}</ANALISADO>
    """

    prompt1 = f"""    Compare o conteúdo de <ORIGINAL></ORIGINAL>, ao qual será chamado daqui em diante de "T.O.", com o conteúdo de <ANALISADO></ANALISADO>, ao qual será chamado daqui em diante de "T.A.", e escreva em formato MARKDOWN respondendo as perguntas numeradas que estão em <PERGUNTAS></PERGUNTAS>.

<PERGUNTAS>
    1) "T.O." e "T.A." fazem solicitações a seus destinatários? Quais são essas solicitações? E qual o beneficiário de cada uma das solicitações?
    2) Desconsiderando autores e destinatários, focando na solicitação e no beneficiário, qual grau de semelhança, de zero a um, poderia ser atribuído entre "T.O." e "T.A."?
</PERGUNTAS>

<ORIGINAL>{original}</ORIGINAL>

<ANALISADO>{analisado}</ANALISADO>
    """

    prompt2 = f"""    Assuma a personalidade de um especialista em plágio de documentos. É muito importante que em sua resposta nunca seja usada a palavra "plágio", em vez disso, se necessário, utilize a palavra similaridade.
    Compare o conteúdo de <ORIGINAL></ORIGINAL>, ao qual será chamado daqui em diante de "T.O.", com o conteúdo de <ANALISADO></ANALISADO>, ao qual será chamado daqui em diante de "T.A." e escreva em formato MARKDOWN respondendo as perguntas numeradas que estão em <PERGUNTAS></PERGUNTAS>. Seja Conciso.

<PERGUNTAS>
    1) Existe semelhança temática entre "T.O." e "T.A."? Quais são estas semelhanças? É o mesmo tema (frize)?
    2) Existe semelhança nas solicitações feita em "T.O." e "T.A."? Quais são estas semelhanças?
    3) A solicitações feitas é uma melhoria para algum lugar, seja esse lugar um logradouro público, um bairro, um prédio comercial, um prédio público, ou ainda instituições públicas e/ou privadas. Que lugar é este? É o mesmo lugar?
    4) Alguma consideração a fazer quanto a similaridade entre "T.O." e "T.A."?
    5) Desconsiderando autores e destinatários, focando na solicitação e no beneficiário, qual grau de semelhança, de zero a um, poderia ser atribuído entre "T.O." e "T.A."?
</PERGUNTAS>

<ORIGINAL>{original}</ORIGINAL>

<ANALISADO>{analisado}</ANALISADO>
    """

    prompts = [prompt0, prompt1, prompt2]

    prompt = prompts[num_prompt]
    while '  ' in prompt:
        prompt = prompt.replace('  ', ' ')

    return prompt


In [25]:
from sapl.materia.models import MateriaLegislativa

originais = MateriaLegislativa.objects.filter(numero=370, tipo_id=3, ano=2025).order_by('id')
analisadas = MateriaLegislativa.objects.filter(numero=46, tipo_id=3, ano=2025).order_by('id')

fez_um_doc = False
for original in originais:
    for analisada in analisadas:
        if original == analisada:
            continue

        if not set(original.autores.all()) - set(analisada.autores.all()):
            continue

        fez_um_doc = True

        print(original, '≃', analisada)

        doc_original = pymupdf.open(original.texto_original.path)
        text_original = ' '.join([page.get_text() for page in doc_original])
        text_original = clean_text(text_original)

        doc_analisada = pymupdf.open(analisada.texto_original.path)
        text_analisada = ' '.join([page.get_text() for page in doc_analisada])
        text_analisada = clean_text(text_analisada)

        prompt = make_prompt(0, text_original, text_analisada, original.epigrafe_short, analisada.epigrafe_short)

        answer = model.generate_content(prompt)
        display(Markdown(answer.text))

        if fez_um_doc:
            break

    if fez_um_doc:
        break


Requerimento nº 370 de 2025 ≃ Requerimento nº 46 de 2025


### Análise Comparativa dos Textos Legislativos

#### Pedidos de Benefícios e Localidades

-   **Os textos estão pedindo o mesmo benefício para a mesma localidade?**

    Não.

-   **Descrição Sucinta dos Pedidos:**

    *   **REQ 370/2025:** Solicita estudos de viabilidade para a transição da posse precária de um lote no distrito da Estância para a categoria dos Motoristas de Transporte Escolar da Escola Municipal Romualda de Barros, visando a construção de um ponto de apoio para seus veículos.
        *   **Beneficiários:** Motoristas de Transporte Escolar da Escola Municipal Romualda de Barros.
        *   **Localidade:** Distrito da Estância, Jataí-GO.

    *   **REQ 046/2025:** Solicita ao Poder Executivo um estudo de viabilidade técnica para a construção de uma Escola em tempo integral no município.
        *   **Beneficiários:** Alunos do município de Jataí-GO.
        *   **Localidade:** Município de Jataí-GO.

#### Semelhança Percentual

A semelhança entre os documentos, focando na solicitação e no beneficiário, é de aproximadamente \[\[ 10% ]].
